## Passo a passo do projeto

1 - Importar e visualizar a base de dados 

2 - Criar o navegador

3 - Para cada item da base de dados:

    3.1 - Procurar o produto no google shopping e no buscapé
    
    3.2 - Verificar se os produtos nos sites estão dentro da faixa de preço
    
4 - Salvar as ofertas que correspondem as condições da pesquisa em um dataframe 

5 - Exportar para excel

6 - Enviar o novo dataframe por email

In [1]:
# 1 - Importar e visualizar a base de dados 
import pandas as pd 
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000.0,3500.0
1,rtx 3060,zota galax,4000.0,4500.0


#### Criação da função para realizar a busca no google

In [2]:
# 3.1 - Pesquisando o produto no google shopping
navegador = webdriver.Chrome()
def busca_google_shopping(navegador, produto, termos_banidos, preco_min, preco_max):
    navegador.get("https://www.google.com/")
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_produto = produto.split(' ')
    preco_min = float(preco_min)
    preco_max = float(preco_max)
    navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
    abas_google = navegador.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for item in abas_google:
        if 'Shopping' in item.text:
            item.click()
            break
    # 3.1.1 - Pegando os preços dos produtos
    lista_oferta = []
    lista_resultados = navegador.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')
    for resultado in lista_resultados:
        nome_produto = resultado.find_element(By.CLASS_NAME, 'Xjkr3b').text
        nome_produto = nome_produto.lower()
        # verificando as especificações do produto
        tem_termos_banidos = False
        for termo in lista_termos_banidos:
            if termo in nome_produto:
                tem_termos_banidos = True
        tem_todos_termos = True
        for termo in lista_termos_produto:
            if termo not in nome_produto:
                tem_todos_termos = False

        if not tem_termos_banidos and tem_todos_termos:
            try:
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
                preco = float(preco)
            except:
                continue
            
            if preco_min <= preco <= preco_max:
                nome_loja = resultado.find_element(By.CLASS_NAME, 'aULzUe')
                link_loja = nome_loja.find_element(By.XPATH, '..')
                link = link_loja.get_attribute('href')
                lista_oferta.append((nome_produto, preco, link))
    return (lista_oferta)

#### Criação da tabela de ofertas com os resultados enconstrados

In [3]:
tabela_ofertas = pd.DataFrame()
for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, 'Nome']
    preco_min = tabela_produtos.loc[linha, 'Preço mínimo']
    preco_max = tabela_produtos.loc[linha, 'Preço máximo']
    termos_banidos = tabela_produtos.loc[linha, 'Termos banidos']
    lista_ofertas_google_shopping = busca_google_shopping(navegador, produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_google_shopping:
        tabela_ofertas_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['Produto', 'Preço', 'Link da loja'])
        tabela_ofertas = tabela_ofertas.append( tabela_ofertas_google_shopping)
    else:
        tabela_ofertas_google_shopping = None
busca_google_shopping(navegador, produto, termos_banidos, preco_min, preco_max)

[('placa de video msi rtx 3060 12gb gddr6 gaming z trio oc 912-v390-245',
  4259.9,
  'https://www.google.com/url?url=https://www.americanas.com.br/produto/6044876400%3Fopn%3DYSMESP%26offerId%3D633b76229064f2befba74e55%26srsltid%3DAR5OiO0psPjF_s9_fHR6eqpSxSwTkY1ZqyBwk4h7J_-H1prza0QSSjxtJs8&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwiqpdPzsNj6AhW2LrkGHR3vB5MQguUECJcP&usg=AOvVaw27Dcjzhg3QE5NLzETxXvPw'),
 ('computador pichau gamer, intel i5-10400f, geforce rtx 3060 12gb, 8gb ddr4, ssd 240gb',
  4399.97,
  'https://www.google.com/url?url=https://www.pichau.com.br/computador-pichau-gamer-intel-i5-10400f-geforce-rtx-3060-12gb-8gb-ddr4-ssd-240gb-desmontado&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwiqpdPzsNj6AhW2LrkGHR3vB5MQgOUECNEQ&usg=AOvVaw3by5-Gn_IJ4JMBnSRljEiH'),
 ('placa de video msi geforce rtx 3060 gaming z 3x 12gb gddr6 - 912-v390-245',
  4344.75,
  'https://www.google.com/url?url=https://www.buscape.com.br/lead%3Foid%3D927655733%26sortorder%3D-1%26index%3D%26searchterm%3Dnull%26pagesize%3D-1%26channel%3D

#### Expotando a tabela para excel

In [4]:
display(tabela_ofertas)
tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel('Ofertas.xlsx', index=False)


,Produto,Preço,Link da loja
0,iphone 12 64gb usado,3400.00,https://www.google.com/url?url=https://sp.olx....
1,"iphone 11 64gb preto, com tela de 6,1, 4g e câ...",3212.00,https://www.google.com/url?url=https://www.pon...
2,"apple iphone 12, 64 gb, preto - totalmente des...",3045.00,https://www.google.com/url?url=https://tiendam...
3,"apple iphone 12, 64 gb, verde - totalmente des...",3095.00,https://www.google.com/url?url=https://tiendam...
0,placa de video msi rtx 3060 12gb gddr6 gaming ...,4259.90,https://www.google.com/url?url=https://www.ame...
1,"computador pichau gamer, intel i5-10400f, gefo...",4399.97,https://www.google.com/url?url=https://www.pic...
2,placa de video msi geforce rtx 3060 gaming z 3...,4344.75,https://www.google.com/url?url=https://www.bus...


#### Enviando e-mail com os resultados




























In [ ]:
# verificando se existe alguma oferta na tabela
if len(tabela_ofertas.index) > 0:
    


















